In [1]:
import csv
print(csv.__file__)

import os
import shutil

/Users/soniasharapova/miniconda3/lib/python3.9/csv.py


In [25]:
os.getcwd()
os.getcwd()
os.listdir()
os.chdir("/Volumes/Toshiba_blue")
os.listdir()
os.getcwd()

'/Volumes/Toshiba_blue'

In [26]:
os.chdir("./methylation/methylationFiles/files")
os.getcwd()

'/Volumes/Toshiba_blue/methylation/methylationFiles/files'

In [31]:
directory = os.getcwd()
for root,dirs,files in os.walk(directory):
    files = [f for f in files if not f[0] == '.']
    dirs[:] = [d for d in dirs if not d[0] == '.']
    os.makedirs('./text', exist_ok=True)
    for file in files:
        if file.endswith(".txt"):
            shutil.copy(file, './text', follow_symlinks=True)
            path = directory + '/' + file

KeyboardInterrupt: 

In [ ]:
def create_rna_methylation_sheet(methylation_sheet, gene_data, directory, is_save=True):
    gene_id = get_all_gene_id()
    df = pd.DataFrame(columns=gene_id)
    gene_id = np.intersect1d(gene_data.index.unique().values, gene_id)
    gene_data = gene_data.loc[gene_id]
    for root,dirs,files in os.walk(directory):
        files = [f for f in files if not f[0] == '.']
        dirs[:] = [d for d in dirs if not d[0] == '.']
        for file in files:
            #print(file)
            if file.endswith(".txt"):
                path = directory + '/' + file
                meth_data = pd.read_csv(path, sep="\t", header=None)
                tmp = meth_data[0].apply(lambda x: x[:2])
                cg_pairs_index = tmp[tmp == 'cg'].index
                meth_data = meth_data.loc[cg_pairs_index]
                location = meth_data[0].apply(lambda x: int(x[2:]))
                meth_data.set_index(0, inplace=True)
                map_path = './methylation/methylationFiles/temp/' + file[:-4] + '_rna_methylation_map.csv'
                os.makedirs('./methylation/methylationFiles/temp/', exist_ok=True)  
                if os.path.exists(map_path):
                    rna_methylation_map = pd.read_csv(map_path, index_col=0)
                    func = lambda x: x[1: -1].replace('\n', '').replace('... ', '').replace("'", '').split(' ') if x != '['']' else []
                    for id in gene_id:
                        rna_methylation_map.loc[id, '0'] = func(rna_methylation_map.loc[id, '0'])
                else:
                    rna_methylation_map = pd.DataFrame(index=gene_id, columns=['Indicator'])
                    print('Generating the RNA-Methylation Map...')
                    for id in tqdm.tqdm(gene_id):
                        is_methylated = location.apply(lambda x: np.any(gene_data.loc[id, 'Intervals'].contains(x))).values
                        rna_methylation_map.loc[id, 'Indicator'] = meth_data[is_methylated].index.values
                    rna_methylation_map.to_csv(map_path)
                    
                for index, row in tqdm.tqdm(methylation_sheet.iterrows()):
                    #file = './methylation/methylationFiles/files/' + row['File Name']
                    print(file)
                    meth_data = downloadMathylData('./methylation/methylationFiles/files/' + file)
                    #meth_data = pd.read_table('./methylation/methylationFiles/files/' + row['File Name'], header=None)
                    meth_data = pd.DataFrame(meth_data)
                    #meth_data.set_index(0, inplace=True)
                    #meth_data.fillna(value=0, inplace=True)
                    if len(meth_data) != 485577:
                        continue
                    func = lambda x: np.mean(meth_data.loc[rna_methylation_map.loc[x, '0']].values)
                    df.loc[row['Case ID'], gene_id] = np.vectorize(func)(gene_id)
                df.fillna(value=0, inplace=True)
                print("done tmp")
                
                if is_save:
                    df.to_csv('./methylation/methylationFiles/outfile.csv')
        return df
                